# Displacement Risk from Floods in Bangladesh

This notebook calculates displacement risk from coastal and riverine floods for Bangladesh.

**Workflow:**
1. Load BEM exposure data for Bangladesh
2. Assign CAPRA/CIMA vulnerability functions
3. Load preprocessed flood hazards (coastal and riverine)
4. Compute physical building damage
5. Apply displacement thresholds
6. Calculate displacement impacts and save results

## 1. Setup and Imports

In [ ]:
import copy
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os

# Change to project root directory (parent of scripts/)
os.chdir(Path.cwd().parent if Path.cwd().name == 'scripts' else Path.cwd())
print(f"Working directory: {os.getcwd()}")

# Add scripts directory to path
sys.path.insert(0, str(Path.cwd() / 'scripts'))

from climada.hazard import Hazard
from climada.entity.exposures import Exposures
from climada.engine import ImpactCalc

# Import local modules (adjust paths as needed)
import flood_hazard
import exposure
import vulnerability

Working directory: /Users/simonameiler/Documents/work/03_code/repos/3MIP


In [2]:
# Configuration
COUNTRY_ISO = 'BGD'
COUNTRY_NAME = 'Bangladesh'

# Paths (relative to project root since we changed directory in first cell)
PATH_HAZARD = Path('data/hazard/BGD')
PATH_EXPOSURE = Path('data/exposure/bem_cntry_files')
PATH_RESULTS = Path('results')
PATH_RESULTS.mkdir(exist_ok=True)

# Damage thresholds for displacement
DMG_THRESHS = {'low': 0.3, 'med': 0.5, 'high': 0.7}

print(f"Hazard path: {PATH_HAZARD.absolute()}")
print(f"Hazard file exists: {(PATH_HAZARD / 'flood_coastal_bgd.hdf5').exists()}")

Hazard path: /Users/simonameiler/Documents/work/03_code/repos/3MIP/data/hazard/BGD
Hazard file exists: True


## 2. Load and Prepare Exposure

Load BEM exposure data for Bangladesh and create CLIMADA exposure object.

In [3]:
# Load BEM data for Bangladesh using exposure module functions
print("Loading BEM exposure data for Bangladesh...")

gdf_bem_subcomps = exposure.gdf_from_bem_subcomps('BGD', opt='full')
print(f"Loaded {len(gdf_bem_subcomps)} exposure points")

# Filter out points with very low population
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum > 1]
print(f"After filtering: {len(gdf_bem_subcomps)} exposure points")
print(f"Total population: {gdf_bem_subcomps['valhum'].sum():,.0f}")

# Assign admin1 attributes for potential aggregation
gdf_bem_subcomps = exposure.assign_admin1_attr(gdf_bem_subcomps, exposure.path_admin1_attrs, source='gadm')

gdf_bem_subcomps.head()

Loading BEM exposure data for Bangladesh...
Loaded 3206590 exposure points
After filtering: 2130829 exposure points
Total population: 129,138,100


,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry,admin1
0,153305375,BGD,3,edu_priv,W2,6.555277,0.011006,55.883,0.0,44.117,POINT (88.4375 26.52917),235.0
1,153305375,BGD,3,edu_priv,W,2.185092,0.003669,55.883,0.0,44.117,POINT (88.4375 26.52917),235.0
2,153305375,BGD,3,edu_priv,UFB3,46.979429,0.078874,55.883,0.0,44.117,POINT (88.4375 26.52917),235.0
3,153305375,BGD,3,edu_priv,INF,17.480769,0.029348,55.883,0.0,44.117,POINT (88.4375 26.52917),235.0
4,153305375,BGD,3,edu_priv,A2,72.108082,0.121062,55.883,0.0,44.117,POINT (88.4375 26.52917),235.0


## 3. Assign Impact Functions (Vulnerability)

Assign CAPRA/CIMA impact functions based on building types.

In [4]:
# Assign CAPRA/CIMA impact function IDs based on building taxonomy
gdf_bem_subcomps['impf_FL'] = gdf_bem_subcomps['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_CIMA)

print("Impact function IDs assigned")
print(gdf_bem_subcomps['impf_FL'].value_counts())

Impact function IDs assigned
impf_FL
14    769462
12    469672
6     463786
15    414411
3      13498
Name: count, dtype: int64


In [5]:
# Check and clean coordinates before creating Exposure object
print("Checking coordinates...")

# Extract coordinates from geometry
gdf_bem_subcomps['longitude'] = gdf_bem_subcomps.geometry.x
gdf_bem_subcomps['latitude'] = gdf_bem_subcomps.geometry.y

# Check for NaN values
nan_lat = gdf_bem_subcomps['latitude'].isna().sum()
nan_lon = gdf_bem_subcomps['longitude'].isna().sum()
print(f"NaN values - Latitude: {nan_lat}, Longitude: {nan_lon}")

# Remove rows with NaN coordinates
if nan_lat > 0 or nan_lon > 0:
    gdf_bem_subcomps = gdf_bem_subcomps[~gdf_bem_subcomps['latitude'].isna() & ~gdf_bem_subcomps['longitude'].isna()]
    print(f"Removed {nan_lat + nan_lon} rows with NaN coordinates")

# Check for infinite values
inf_lat = np.isinf(gdf_bem_subcomps['latitude']).sum()
inf_lon = np.isinf(gdf_bem_subcomps['longitude']).sum()
print(f"Inf values - Latitude: {inf_lat}, Longitude: {inf_lon}")

# Remove rows with infinite coordinates
if inf_lat > 0 or inf_lon > 0:
    gdf_bem_subcomps = gdf_bem_subcomps[~np.isinf(gdf_bem_subcomps['latitude']) & ~np.isinf(gdf_bem_subcomps['longitude'])]
    print(f"Removed {inf_lat + inf_lon} rows with infinite coordinates")

# Create CLIMADA Exposure object
exp = Exposures(gdf_bem_subcomps.copy())
exp.value_unit = 'building_unit'
exp.gdf['value'] = 1

print(f"\nExposure object created:")
print(f"  Points: {len(exp.gdf)}")
print(f"  Lat range: [{gdf_bem_subcomps['latitude'].min():.2f}, {gdf_bem_subcomps['latitude'].max():.2f}]")
print(f"  Lon range: [{gdf_bem_subcomps['longitude'].min():.2f}, {gdf_bem_subcomps['longitude'].max():.2f}]")
print(f"  Total population: {exp.gdf['valhum'].sum():,.0f}")
print(f"  Available columns: {list(exp.gdf.columns)}")

Checking coordinates...
NaN values - Latitude: 168, Longitude: 168
Removed 336 rows with NaN coordinates
Inf values - Latitude: 0, Longitude: 0

Exposure object created:
  Points: 2130661
  Lat range: [20.75, 26.62]
  Lon range: [88.03, 92.63]
  Total population: 129,129,257
  Available columns: ['id_1x', 'iso3', 'cpx', 'sector', 'se_seismo', 'valhum', 'valfis', 'bd_1_floor', 'bd_2_floor', 'bd_3_floor', 'admin1', 'impf_FL', 'geometry', 'value']


## 4. Load Impact Functions

Load CAPRA/CIMA impact function set for flood vulnerability.

In [6]:
# Load CAPRA/CIMA impact function set
IMPF_SET_FL_CIMA = vulnerability.IMPF_SET_FL_CIMA

print("Impact functions loaded")
print(f"Available function IDs: {IMPF_SET_FL_CIMA.get_ids()}")

Impact functions loaded
Available function IDs: {'FL': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}


## 5. Load Preprocessed Flood Hazards

Load coastal and riverine flood hazards that were preprocessed for Bangladesh.

In [7]:
# Load coastal flood hazard
print("Loading coastal flood hazard...")
haz_coastal = flood_hazard.load_preprocessed_hazard('coastal', PATH_HAZARD)

print(f"\nCoastal flood hazard:")
print(f"  Events: {haz_coastal.size}")
print(f"  Return periods: {haz_coastal.event_name}")
print(f"  Centroids: {len(haz_coastal.centroids.lat)}")
print(f"  Max intensities: {haz_coastal.intensity.max(axis=1).toarray().flatten()}")

Loading coastal flood hazard...
Loading preprocessed coastal flood hazard from data/hazard/BGD/flood_coastal_bgd.hdf5...
Loaded 9 events with 428072 centroids

Coastal flood hazard:
  Events: 9
  Return periods: ['RP_2', 'RP_5', 'RP_10', 'RP_25', 'RP_50', 'RP_100', 'RP_250', 'RP_500', 'RP_1000']
  Centroids: 428072
  Max intensities: [ 5.95700073  7.46814823  7.94548512  8.66280365  9.18778419  9.66451359
 10.29220676 10.76616287 11.23977661]


In [8]:
# Load riverine flood hazard
print("Loading riverine flood hazard...")
haz_riverine = flood_hazard.load_preprocessed_hazard('riverine', PATH_HAZARD)

print(f"\nRiverine flood hazard:")
print(f"  Events: {haz_riverine.size}")
print(f"  Return periods: {haz_riverine.event_name}")
print(f"  Centroids: {len(haz_riverine.centroids.lat)}")
print(f"  Max intensities: {haz_riverine.intensity.max(axis=1).toarray().flatten()}")

Loading riverine flood hazard...
Loading preprocessed riverine flood hazard from data/hazard/BGD/flood_riverine_bgd.hdf5...
Loaded 9 events with 428072 centroids

Riverine flood hazard:
  Events: 9
  Return periods: ['RP_2', 'RP_5', 'RP_10', 'RP_25', 'RP_50', 'RP_100', 'RP_250', 'RP_500', 'RP_1000']
  Centroids: 428072
  Max intensities: [ 0.          7.61462498 11.19643974 15.16551399 17.76284409 20.15526962
 23.09294891 25.17805862 27.13647079]


In [9]:
# Extract return periods for later use
rps = [int(name.split('_')[1]) for name in haz_coastal.event_name]
print(f"Return periods: {rps}")

Return periods: [2, 5, 10, 25, 50, 100, 250, 500, 1000]


## 6. Compute Physical Impacts

Calculate building damage for each flood type and return period.

Note, the results are fractional damage values. So, impact.at_event yields the sum of fractional building damages.

In [10]:
# Compute impact for coastal floods
print("Computing coastal flood impacts...")
imp_coastal = ImpactCalc(exp, IMPF_SET_FL_CIMA, haz_coastal).impact(save_mat=True)

print(f"\nCoastal flood impacts:")
print(f"  Impact matrix shape: {imp_coastal.imp_mat.shape}")
print(f"  Annual average impact: {imp_coastal.aai_agg:,.0f}")
print(f"  Impact per event: {imp_coastal.at_event}")

Computing coastal flood impacts...

Coastal flood impacts:
  Impact matrix shape: (9, 2130661)
  Annual average impact: 42,943
  Impact per event: [ 23117.26381223  57072.83107643  80095.2979831  121817.89885174
 160141.53088296 197703.11573243 249850.65611768 289936.89942088
 328501.62951724]


In [11]:
# Compute impact for riverine floods
print("Computing riverine flood impacts...")
imp_riverine = ImpactCalc(exp, IMPF_SET_FL_CIMA, haz_riverine).impact(save_mat=True)

print(f"\nRiverine flood impacts:")
print(f"  Impact matrix shape: {imp_riverine.imp_mat.shape}")
print(f"  Annual average impact: {imp_riverine.aai_agg:,.0f}")
print(f"  Impact per event: {imp_riverine.at_event}")

Computing riverine flood impacts...

Riverine flood impacts:
  Impact matrix shape: (9, 2130661)
  Annual average impact: 16,523
  Impact per event: [     0.          25201.4197143   44987.95371515  71362.12533735
  92853.79833643 115364.48328162 146313.08323375 170091.28374459
 193372.61576027]


## 7. Apply Displacement Thresholds

Convert building damage to displacement based on damage thresholds.

Meaning, we assign a damage fraction threshold and then assume all people in buildings that are above the threshold displaced.

In [12]:
def apply_displacement_thresholds(imp_mat, thresholds):
    """
    Apply damage thresholds to impact matrix.
    Returns dict of boolean matrices for each threshold.
    """
    bool_mats = {}
    for thresh_name, thresh_val in thresholds.items():
        bool_mats[thresh_name] = imp_mat > thresh_val
    return bool_mats

# Apply thresholds to coastal floods
print("Applying displacement thresholds to coastal floods...")
displ_coastal = apply_displacement_thresholds(imp_coastal.imp_mat, DMG_THRESHS)

# Apply thresholds to riverine floods  
print("Applying displacement thresholds to riverine floods...")
displ_riverine = apply_displacement_thresholds(imp_riverine.imp_mat, DMG_THRESHS)

print("\nThresholds applied successfully")

Applying displacement thresholds to coastal floods...
Applying displacement thresholds to riverine floods...

Thresholds applied successfully


## 8. Calculate Displacement Impacts

Multiply boolean displacement matrices by population to get displaced people.

In [13]:
def calculate_displacement(bool_mat, population):
    """
    Calculate displaced population from boolean matrix.
    Returns displacement per event.
    
    bool_mat: sparse matrix (n_events, n_exposure_points)
    population: Series with population per exposure point
    """
    # Matrix multiplication: (events, points) × (points,) = (events,)
    # This sums (boolean * population) for each event
    displacement = bool_mat.dot(population.values)
    return displacement

# Calculate displacement for coastal floods
results_coastal = {}
for thresh in DMG_THRESHS.keys():
    displ_per_event = calculate_displacement(displ_coastal[thresh], exp.gdf['valhum'])
    results_coastal[thresh] = displ_per_event
    print(f"Coastal {thresh} threshold - RP100 displacement: {displ_per_event[5]:,.0f}")

# Calculate displacement for riverine floods
results_riverine = {}
for thresh in DMG_THRESHS.keys():
    displ_per_event = calculate_displacement(displ_riverine[thresh], exp.gdf['valhum'])
    results_riverine[thresh] = displ_per_event
    print(f"Riverine {thresh} threshold - RP100 displacement: {displ_per_event[5]:,.0f}")

Coastal low threshold - RP100 displacement: 10,287,962
Coastal med threshold - RP100 displacement: 8,526,897
Coastal high threshold - RP100 displacement: 6,222,500
Riverine low threshold - RP100 displacement: 8,526,230
Riverine med threshold - RP100 displacement: 5,922,603
Riverine high threshold - RP100 displacement: 3,123,252


## 9. Calculate Annual Expected Displacement (AED)

Calculate expected annual displacement by integrating over return periods.

In [14]:
def calculate_aed(displacement_per_event, frequencies):
    """
    Calculate annual expected displacement.
    """
    return np.sum(displacement_per_event * frequencies)

frequencies = haz_coastal.frequency

# Calculate AED for coastal floods
aed_coastal = {}
for thresh in DMG_THRESHS.keys():
    aed_coastal[thresh] = calculate_aed(results_coastal[thresh], frequencies)
    print(f"Coastal {thresh} threshold AED: {aed_coastal[thresh]:,.0f} people/year")

# Calculate AED for riverine floods
aed_riverine = {}
for thresh in DMG_THRESHS.keys():
    aed_riverine[thresh] = calculate_aed(results_riverine[thresh], frequencies)
    print(f"Riverine {thresh} threshold AED: {aed_riverine[thresh]:,.0f} people/year")

Coastal low threshold AED: 1,820,114 people/year
Coastal med threshold AED: 1,486,475 people/year
Coastal high threshold AED: 1,065,657 people/year
Riverine low threshold AED: 929,625 people/year
Riverine med threshold AED: 661,541 people/year
Riverine high threshold AED: 455,579 people/year


## 10.1 Prepare and Save Results (Admin 0)

Create dataframes with results and save to CSV.

In [15]:
# Create results dataframe for coastal floods
df_coastal = pd.DataFrame({
    'return_period': rps,
    'frequency': frequencies
})

for thresh in DMG_THRESHS.keys():
    df_coastal[f'displacement_{thresh}'] = results_coastal[thresh]

# Add AED row
aed_row = {'return_period': 'AED', 'frequency': np.nan}
for thresh in DMG_THRESHS.keys():
    aed_row[f'displacement_{thresh}'] = aed_coastal[thresh]
df_coastal = pd.concat([df_coastal, pd.DataFrame([aed_row])], ignore_index=True)

print("\nCoastal flood displacement results:")
print(df_coastal)


Coastal flood displacement results:
  return_period  frequency  displacement_low  displacement_med  \
0             2      0.500      8.220780e+05      6.666985e+05   
1             5      0.200      2.291444e+06      1.867060e+06   
2            10      0.100      3.525953e+06      2.823392e+06   
3            25      0.040      5.678444e+06      4.675372e+06   
4            50      0.020      8.065134e+06      6.633137e+06   
5           100      0.010      1.028796e+07      8.526897e+06   
6           250      0.004      1.348890e+07      1.156487e+07   
7           500      0.002      1.665581e+07      1.441778e+07   
8          1000      0.001      1.960318e+07      1.733330e+07   
9           AED        NaN      1.820114e+06      1.486475e+06   

   displacement_high  
0       4.815442e+05  
1       1.345191e+06  
2       2.009764e+06  
3       3.277443e+06  
4       4.795887e+06  
5       6.222500e+06  
6       8.261108e+06  
7       1.023062e+07  
8       1.212435e+07  
9     

In [16]:
# Create results dataframe for riverine floods
df_riverine = pd.DataFrame({
    'return_period': rps,
    'frequency': frequencies
})

for thresh in DMG_THRESHS.keys():
    df_riverine[f'displacement_{thresh}'] = results_riverine[thresh]

# Add AED row
aed_row = {'return_period': 'AED', 'frequency': np.nan}
for thresh in DMG_THRESHS.keys():
    aed_row[f'displacement_{thresh}'] = aed_riverine[thresh]
df_riverine = pd.concat([df_riverine, pd.DataFrame([aed_row])], ignore_index=True)

print("\nRiverine flood displacement results:")
print(df_riverine)


Riverine flood displacement results:
  return_period  frequency  displacement_low  displacement_med  \
0             2      0.500      0.000000e+00      0.000000e+00   
1             5      0.200      1.291632e+06      1.011106e+06   
2            10      0.100      2.172145e+06      1.594788e+06   
3            25      0.040      4.246278e+06      2.604610e+06   
4            50      0.020      6.083105e+06      3.617706e+06   
5           100      0.010      8.526230e+06      5.922603e+06   
6           250      0.004      1.028840e+07      8.451750e+06   
7           500      0.002      1.151349e+07      9.710770e+06   
8          1000      0.001      1.312784e+07      1.084802e+07   
9           AED        NaN      9.296249e+05      6.615411e+05   

   displacement_high  
0       0.000000e+00  
1       6.821588e+05  
2       1.224269e+06  
3       1.895893e+06  
4       2.324439e+06  
5       3.123252e+06  
6       5.419664e+06  
7       6.708775e+06  
8       8.067217e+06  
9    

In [17]:
# Save results
output_coastal = PATH_RESULTS / f'{COUNTRY_ISO}_displacement_coastal.csv'
output_riverine = PATH_RESULTS / f'{COUNTRY_ISO}_displacement_riverine.csv'

df_coastal.to_csv(output_coastal, index=False)
df_riverine.to_csv(output_riverine, index=False)

print(f"\nResults saved:")
print(f"  Coastal: {output_coastal}")
print(f"  Riverine: {output_riverine}")


Results saved:
  Coastal: results/BGD_displacement_coastal.csv
  Riverine: results/BGD_displacement_riverine.csv


## 10.2 Calculate Results per Admin1 Region

Aggregate displacement results by administrative level 1 regions.

In [18]:
def calculate_displacement_by_admin1(bool_mat, population, admin1, event_names):
    """
    Calculate displaced population per admin1 region for each event.
    
    Parameters:
    -----------
    bool_mat : sparse matrix (n_events, n_exposure_points)
        Boolean matrix indicating displacement
    population : Series
        Population per exposure point
    admin1 : Series
        Admin1 ID per exposure point
    event_names : list
        Event names for columns
    
    Returns:
    --------
    DataFrame with admin1 regions as index and events as columns
    """
    # Get unique admin1 regions
    admin1_regions = sorted(admin1.unique())
    
    # Create result dataframe
    results = pd.DataFrame(index=admin1_regions, columns=event_names)
    
    # Calculate displacement for each admin1 region
    for region in admin1_regions:
        # Get mask for this admin1 region
        mask = (admin1 == region).values
        
        # Extract population for this region
        pop_region = population.values[mask]
        
        # Extract boolean matrix for this region (subset of columns)
        bool_region = bool_mat[:, mask]
        
        # Calculate displacement per event for this region
        displacement = bool_region.dot(pop_region)
        
        results.loc[region] = displacement
    
    return results.astype(float)

print("Function defined for admin1 aggregation")

Function defined for admin1 aggregation


In [19]:
# Calculate displacement by admin1 for coastal floods
print("Calculating coastal flood displacement by admin1 region...")

results_coastal_admin1 = {}
for thresh in DMG_THRESHS.keys():
    df_admin1 = calculate_displacement_by_admin1(
        displ_coastal[thresh], 
        exp.gdf['valhum'],
        exp.gdf['admin1'],
        haz_coastal.event_name
    )
    results_coastal_admin1[thresh] = df_admin1
    
print(f"Coastal floods - Admin1 regions: {len(results_coastal_admin1['med'])}")
print("\nSample (medium threshold):")
print(results_coastal_admin1['med'].head())

Calculating coastal flood displacement by admin1 region...
Coastal floods - Admin1 regions: 13

Sample (medium threshold):
                RP_2          RP_5         RP_10         RP_25         RP_50  \
229.0  146084.007444  3.233216e+05  5.620739e+05  1.236598e+06  1.916728e+06   
230.0   26390.493357  1.347803e+05  4.052824e+05  8.196975e+05  1.193959e+06   
231.0  110005.547716  2.257273e+05  3.481355e+05  5.811048e+05  1.135659e+06   
232.0  384218.426140  1.183231e+06  1.507072e+06  2.036836e+06  2.385022e+06   
233.0       0.000000  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

             RP_100        RP_250        RP_500       RP_1000  
229.0  2.617610e+06  3.586673e+06  4.310110e+06  4.916428e+06  
230.0  1.632806e+06  2.432220e+06  3.025678e+06  3.681696e+06  
231.0  1.528300e+06  2.297973e+06  3.474285e+06  4.683454e+06  
232.0  2.744821e+06  3.178001e+06  3.462481e+06  3.758910e+06  
233.0  9.577039e+02  8.645068e+03  2.986011e+04  6.653492e+04  


In [20]:
# Calculate displacement by admin1 for riverine floods
print("Calculating riverine flood displacement by admin1 region...")

results_riverine_admin1 = {}
for thresh in DMG_THRESHS.keys():
    df_admin1 = calculate_displacement_by_admin1(
        displ_riverine[thresh], 
        exp.gdf['valhum'],
        exp.gdf['admin1'],
        haz_riverine.event_name
    )
    results_riverine_admin1[thresh] = df_admin1
    
print(f"Riverine floods - Admin1 regions: {len(results_riverine_admin1['med'])}")
print("\nSample (medium threshold):")
print(results_riverine_admin1['med'].head())

Calculating riverine flood displacement by admin1 region...
Riverine floods - Admin1 regions: 13

Sample (medium threshold):
       RP_2           RP_5          RP_10         RP_25         RP_50  \
229.0   0.0       0.000000       0.000000  6.704434e+04  6.874337e+04   
230.0   0.0  540897.552819  835145.177380  1.169257e+06  1.556383e+06   
231.0   0.0  151953.686262  233623.718951  3.917374e+05  4.887475e+05   
232.0   0.0   23361.268273   42567.202673  6.669573e+04  8.781359e+04   
233.0   0.0   21269.386492   41222.135553  1.369791e+05  1.977548e+05   

             RP_100        RP_250        RP_500       RP_1000  
229.0  1.004910e+05  1.442000e+05  1.546117e+05  1.688329e+05  
230.0  1.844730e+06  2.702302e+06  2.890021e+06  3.211165e+06  
231.0  1.308620e+06  1.994437e+06  2.404365e+06  2.604819e+06  
232.0  1.022222e+05  1.224132e+05  1.359346e+05  1.480143e+05  
233.0  3.644469e+05  6.741603e+05  7.909338e+05  9.037725e+05  


In [21]:
# Calculate AED per admin1 region
print("Calculating AED per admin1 region...")

# Coastal floods
aed_coastal_admin1 = {}
for thresh in DMG_THRESHS.keys():
    df = results_coastal_admin1[thresh].copy()
    # Calculate AED per admin1 (displacement × frequency, summed over events)
    df['AED'] = (df * frequencies).sum(axis=1)
    aed_coastal_admin1[thresh] = df

# Riverine floods
aed_riverine_admin1 = {}
for thresh in DMG_THRESHS.keys():
    df = results_riverine_admin1[thresh].copy()
    df['AED'] = (df * frequencies).sum(axis=1)
    aed_riverine_admin1[thresh] = df

print("\nCoastal floods AED by admin1 (medium threshold):")
print(aed_coastal_admin1['med'][['RP_2', 'RP_100', 'RP_1000', 'AED']].head())

print("\nRiverine floods AED by admin1 (medium threshold):")
print(aed_riverine_admin1['med'][['RP_2', 'RP_100', 'RP_1000', 'AED']].head())

Calculating AED per admin1 region...

Coastal floods AED by admin1 (medium threshold):
                RP_2        RP_100       RP_1000            AED
229.0  146084.007444  2.617610e+06  4.916428e+06  335771.639822
230.0   26390.493357  1.632806e+06  3.681696e+06  173136.615511
231.0  110005.547716  1.528300e+06  4.683454e+06  217026.089707
232.0  384218.426140  2.744821e+06  3.758910e+06  759480.448764
233.0       0.000000  9.577039e+02  6.653492e+04     170.412455

Riverine floods AED by admin1 (medium threshold):
       RP_2        RP_100       RP_1000            AED
229.0   0.0  1.004910e+05  1.688329e+05    6116.406856
230.0   0.0  1.844730e+06  3.211165e+06  307839.682697
231.0   0.0  1.308620e+06  2.604819e+06  107675.049616
232.0   0.0  1.022222e+05  1.480143e+05   15284.833091
233.0   0.0  3.644469e+05  9.037725e+05   26637.102346


## 10.3 Rename Admin1 Regions to Readable Names

Map admin1 FID codes to GADM region names for better interpretability.

In [26]:
# Function to rename admin1 indices in dataframes
def rename_admin1_index(df, fid_to_name_map):
    """
    Rename admin1 FID codes to readable GADM region names.
    
    Parameters:
    -----------
    df : DataFrame
        DataFrame with admin1 FID codes as index
    fid_to_name_map : dict
        Mapping from FID to GID_1 names
    
    Returns:
    --------
    DataFrame with renamed index, filtered to only mapped regions
    """
    df_renamed = df.copy()
    
    # Check if index is already renamed (string type)
    if df_renamed.index.dtype == 'object' or df_renamed.index.dtype.name.startswith('str'):
        print("  Index already appears to be renamed, skipping...")
        return df_renamed
    
    # Convert index to int if needed (handle -999 for admin0)
    df_renamed.index = df_renamed.index.astype(float).astype(int)
    
    # Filter to only regions that are in the mapping (removes foreign regions)
    df_renamed = df_renamed[df_renamed.index.isin(fid_to_name_map.keys())]
    
    # Map FID to GID_1
    df_renamed.index = df_renamed.index.map(fid_to_name_map)
    df_renamed.index.name = 'admin1_name'
    
    return df_renamed

print("Function defined for renaming admin1 regions")

Function defined for renaming admin1 regions


In [27]:
# Load GADM attribute table and create FID to GID_1 mapping
path_gadm_attrs = Path('data/shapefiles/attribute_table_gadm41.csv')

if path_gadm_attrs.exists():
    gadm_attrs = pd.read_csv(path_gadm_attrs)
    
    # Filter for Bangladesh admin1 regions
    gadm_bgd = gadm_attrs[gadm_attrs['GID_1'].str.startswith('BGD.', na=False)]
    
    # Create FID to GID_1 mapping
    fid_to_gid = dict(zip(gadm_bgd['fid'], gadm_bgd['GID_1']))
    
    print(f"Loaded {len(fid_to_gid)} admin1 regions for Bangladesh")
    print("Sample mappings:")
    for fid, gid in list(fid_to_gid.items())[:5]:
        print(f"  {fid} -> {gid}")
else:
    print(f"Warning: GADM attribute file not found at {path_gadm_attrs}")
    fid_to_gid = {}

Loaded 8 admin1 regions for Bangladesh
Sample mappings:
  229 -> BGD.1_1
  230 -> BGD.2_1
  231 -> BGD.3_1
  232 -> BGD.4_1
  233 -> BGD.8_1


In [28]:
# Apply renaming to all admin1 dataframes
if fid_to_gid:
    # Rename coastal admin1 results
    for thresh in DMG_THRESHS.keys():
        aed_coastal_admin1[thresh] = rename_admin1_index(aed_coastal_admin1[thresh], fid_to_gid)
        print(f"Renamed aed_coastal_admin1['{thresh}']")
    
    # Rename riverine admin1 results
    for thresh in DMG_THRESHS.keys():
        aed_riverine_admin1[thresh] = rename_admin1_index(aed_riverine_admin1[thresh], fid_to_gid)
        print(f"Renamed aed_riverine_admin1['{thresh}']")
    
    print("\nAll admin1 results renamed successfully!")
else:
    print("Skipping renaming - no mapping available")

Renamed aed_coastal_admin1['low']
Renamed aed_coastal_admin1['med']
Renamed aed_coastal_admin1['high']
Renamed aed_riverine_admin1['low']
Renamed aed_riverine_admin1['med']
Renamed aed_riverine_admin1['high']

All admin1 results renamed successfully!


In [ ]:
# Save admin1 results with readable region names (one file per threshold)
print("Saving admin1 results...")

# Coastal flood admin1 results - one file per threshold
for thresh in DMG_THRESHS.keys():
    output_file = PATH_RESULTS / f'{COUNTRY_ISO}_displacement_coastal_admin1_{thresh}.csv'
    aed_coastal_admin1[thresh].to_csv(output_file)
    print(f"Saved {output_file.name}")

# Riverine flood admin1 results - one file per threshold
for thresh in DMG_THRESHS.keys():
    output_file = PATH_RESULTS / f'{COUNTRY_ISO}_displacement_riverine_admin1_{thresh}.csv'
    aed_riverine_admin1[thresh].to_csv(output_file)
    print(f"Saved {output_file.name}")

Saving admin1 results...
Saved BGD_displacement_coastal_admin1_low.csv
Saved BGD_displacement_coastal_admin1_med.csv
Saved BGD_displacement_coastal_admin1_high.csv
Saved BGD_displacement_riverine_admin1_low.csv
Saved BGD_displacement_riverine_admin1_med.csv
Saved BGD_displacement_riverine_admin1_high.csv

Admin1 results saved to: results


## 10.4 Summary: Top regions by displacement

In [31]:
# Display top 5 admin1 regions by AED for each flood type and threshold
print("=" * 80)
print("TOP 5 ADMIN1 REGIONS BY ANNUAL EXPECTED DISPLACEMENT (AED)")
print("=" * 80)

print("\n### COASTAL FLOODS ###\n")
for threshold_name, thresh in [('Low (>0.3m)', 'low'),
                                ('Medium (>0.5m)', 'med'),
                                ('High (>0.7m)', 'high')]:
    print(f"\n{threshold_name} threshold:")
    print("-" * 40)
    top5 = aed_coastal_admin1[thresh]['AED'].sort_values(ascending=False).head(5)
    for idx, val in top5.items():
        print(f"  {idx}: {val:,.0f} people/year")

print("\n" + "=" * 80)
print("\n### RIVERINE FLOODS ###\n")
for threshold_name, thresh in [('Low (>0.3m)', 'low'),
                                ('Medium (>0.5m)', 'med'),
                                ('High (>0.7m)', 'high')]:
    print(f"\n{threshold_name} threshold:")
    print("-" * 40)
    top5 = aed_riverine_admin1[thresh]['AED'].sort_values(ascending=False).head(5)
    for idx, val in top5.items():
        print(f"  {idx}: {val:,.0f} people/year")

print("\n" + "=" * 80)

TOP 5 ADMIN1 REGIONS BY ANNUAL EXPECTED DISPLACEMENT (AED)

### COASTAL FLOODS ###


Low (>0.3m) threshold:
----------------------------------------
  BGD.4_1: 908,040 people/year
  BGD.1_1: 430,120 people/year
  BGD.3_1: 254,335 people/year
  BGD.2_1: 226,124 people/year
  BGD.7_1: 999 people/year

Medium (>0.5m) threshold:
----------------------------------------
  BGD.4_1: 759,480 people/year
  BGD.1_1: 335,772 people/year
  BGD.3_1: 217,026 people/year
  BGD.2_1: 173,137 people/year
  BGD.7_1: 706 people/year

High (>0.7m) threshold:
----------------------------------------
  BGD.4_1: 550,962 people/year
  BGD.1_1: 240,016 people/year
  BGD.3_1: 152,041 people/year
  BGD.2_1: 121,937 people/year
  BGD.7_1: 504 people/year


### RIVERINE FLOODS ###


Low (>0.3m) threshold:
----------------------------------------
  BGD.2_1: 392,639 people/year
  BGD.3_1: 183,977 people/year
  BGD.5_1: 128,405 people/year
  BGD.6_1: 77,862 people/year
  BGD.7_1: 74,989 people/year

Medium (>0.5m) thr

## 10.5 Summary Admin 0

Display final summary of displacement risk for Bangladesh.

In [32]:
print("="*70)
print(f"DISPLACEMENT RISK SUMMARY - {COUNTRY_NAME}")
print("="*70)

print("\nCoastal Floods:")
for thresh in DMG_THRESHS.keys():
    print(f"  {thresh.upper()} threshold ({DMG_THRESHS[thresh]:.1f}): {aed_coastal[thresh]:,.0f} people/year")

print("\nRiverine Floods:")
for thresh in DMG_THRESHS.keys():
    print(f"  {thresh.upper()} threshold ({DMG_THRESHS[thresh]:.1f}): {aed_riverine[thresh]:,.0f} people/year")

print("\nTotal (Combined):")
for thresh in DMG_THRESHS.keys():
    total = aed_coastal[thresh] + aed_riverine[thresh]
    print(f"  {thresh.upper()} threshold: {total:,.0f} people/year")

DISPLACEMENT RISK SUMMARY - Bangladesh

Coastal Floods:
  LOW threshold (0.3): 1,820,114 people/year
  MED threshold (0.5): 1,486,475 people/year
  HIGH threshold (0.7): 1,065,657 people/year

Riverine Floods:
  LOW threshold (0.3): 929,625 people/year
  MED threshold (0.5): 661,541 people/year
  HIGH threshold (0.7): 455,579 people/year

Total (Combined):
  LOW threshold: 2,749,738 people/year
  MED threshold: 2,148,016 people/year
  HIGH threshold: 1,521,236 people/year
